# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [50]:
import warnings
warnings.filterwarnings('ignore')

# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [51]:
city_data = pd.read_csv("../WeatherPy/city_data.csv")
city_data.head(8)

,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,carnarvon,-24.87,113.63,19.00,82,90,8.70,AU,1594961110
1,cape town,-33.93,18.42,8.89,75,0,2.10,ZA,1594961158
2,zemio,5.03,25.14,20.29,96,74,0.13,CF,1594961159
3,hilo,19.73,-155.09,27.00,74,20,2.10,US,1594961159
4,nikolskoye,59.70,30.79,15.56,77,86,4.00,RU,1594961159
5,nanortalik,60.14,-45.24,8.77,60,2,2.53,GL,1594961159
6,hithadhoo,-0.60,73.08,29.05,72,95,4.34,MV,1594961160
7,ribeira grande,38.52,-28.70,20.51,80,36,4.78,PT,1594961160


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [52]:
locations = city_data[["Lat","Lon"]]
humidity = city_data["Humidity"]
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                 dissipating=False, max_intensity=25,
                                 point_radius=2)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

My ideal weather will be between °70-°80F (°21-°26C) with no cloudiness and less than 50% humidity, so my hair can maintain optimal curl.

In [53]:
ideal_temp = city_data.loc[(city_data["Max Temp"] < 26) & (city_data["Max Temp"] > 21) & 
                           (city_data["Cloudiness"] == 0) & (city_data["Humidity"] < 50)]  
ideal_temp

,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
320,hovd,48.01,91.64,21.41,33,0,3.15,MN,1594961229
333,jalu,29.03,21.55,25.45,45,0,3.35,LY,1594961232
362,luderitz,-26.65,15.16,24.00,14,0,9.30,NaN,1594961237
364,cabra,37.47,-4.44,23.33,49,0,1.97,ES,1594961238
439,ulaangom,49.98,92.07,23.71,37,0,2.42,MN,1594961253
442,namibe,-15.20,12.15,21.85,29,0,5.43,AO,1594961253
530,charters towers,-20.10,146.27,22.36,23,0,3.30,AU,1594961272


In [54]:
ideal_temp = ideal_temp.dropna(how='any')
ideal_temp

,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
320,hovd,48.01,91.64,21.41,33,0,3.15,MN,1594961229
333,jalu,29.03,21.55,25.45,45,0,3.35,LY,1594961232
364,cabra,37.47,-4.44,23.33,49,0,1.97,ES,1594961238
439,ulaangom,49.98,92.07,23.71,37,0,2.42,MN,1594961253
442,namibe,-15.20,12.15,21.85,29,0,5.43,AO,1594961253
530,charters towers,-20.10,146.27,22.36,23,0,3.30,AU,1594961272


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [55]:
ideal_temp["Hotel Name"] = ""
hotel_df = ideal_temp

hotel_df

,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
320,hovd,48.01,91.64,21.41,33,0,3.15,MN,1594961229,
333,jalu,29.03,21.55,25.45,45,0,3.35,LY,1594961232,
364,cabra,37.47,-4.44,23.33,49,0,1.97,ES,1594961238,
439,ulaangom,49.98,92.07,23.71,37,0,2.42,MN,1594961253,
442,namibe,-15.20,12.15,21.85,29,0,5.43,AO,1594961253,
530,charters towers,-20.10,146.27,22.36,23,0,3.30,AU,1594961272,


In [71]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lon = row["Lon"]

    # add keyword to params dict
    params["location"]= f"{lat},{lon}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {hotel}.")
    response = requests.get(base_url, params=params).json()
#     print(response)
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest {hotel} hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
     
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 320: .
Closest  hotel is Hotel Khovd.
------------
Retrieving Results for Index 333: .
Closest  hotel is Jalu Hotel.
------------
Retrieving Results for Index 364: .
Closest  hotel is MS Fuente Las Piedras.
------------
Retrieving Results for Index 439: .
Closest  hotel is Achit Nuur Hotel & Restaurant.
------------
Retrieving Results for Index 442: .
Closest  hotel is Hotel Chik Chik Namibe.
------------
Retrieving Results for Index 530: .
Closest  hotel is Cattleman's Rest Motor Inn.
------------


In [77]:
hotel_df

,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
320,hovd,48.01,91.64,21.41,33,0,3.15,MN,1594961229,Hotel Khovd
333,jalu,29.03,21.55,25.45,45,0,3.35,LY,1594961232,Jalu Hotel
364,cabra,37.47,-4.44,23.33,49,0,1.97,ES,1594961238,MS Fuente Las Piedras
439,ulaangom,49.98,92.07,23.71,37,0,2.42,MN,1594961253,Achit Nuur Hotel & Restaurant
442,namibe,-15.20,12.15,21.85,29,0,5.43,AO,1594961253,Hotel Chik Chik Namibe
530,charters towers,-20.10,146.27,22.36,23,0,3.30,AU,1594961272,Cattleman's Rest Motor Inn


In [78]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lon"]]

In [85]:
# Add marker layer ontop of heat map
humidity = hotel_df["Humidity"]
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                 dissipating=False, max_intensity=100,
                                 point_radius=10)

fig.add_layer(heat_layer)

markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Add the Hotel Marker layer to the map
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))